In [1]:
import Test.QuickCheck
import Control.Applicative
import Data.List (elemIndex)
import Data.Monoid
import Test.QuickCheck.Checkers
import Test.QuickCheck.Classes

In [2]:
applicativeId :: (Applicative f, Eq (f a))
    => f a -> Bool
applicativeId m = (pure id <*> m) == m

In [3]:
applicativeCompose :: (Applicative f, Eq (f c))
    => (a -> b) -> (b -> c) -> f a -> Bool
applicativeCompose u v w = (pure (.) <*> pure v <*> pure u <*> w) == (pure v <*> (pure u <*> w))

In [4]:
{--
{-# LANGUAGE AllowAmbiguousTypes #-}

applicativeHomomorphism :: (Applicative f, Eq (f b))
    => f b -> (a -> b) -> a -> Bool
applicativeHomomorphism _ fn x = lhs == pure (fn x)
    where lhs :: f b 
          lhs = (pure fn <*> pure x)
--}

In [5]:
applicativeHomomorphism :: (Applicative f, Eq (f b))
    => f (a -> b) -> (a -> b) -> a -> Bool
applicativeHomomorphism ffn fn x = (ffn <*> pure x) == pure (fn x)

In [6]:
applicativeInterchange :: (Applicative f, Eq (f b))
    => f (a -> b) -> a -> Bool
applicativeInterchange u y = (u <*> (pure y)) == (pure ($ y) <*> u)

<h4>Tupley Type</h4>

In [7]:
data Tupley a b = Tupley {first:: a, second:: b} deriving (Eq, Show)

instance (Arbitrary a, Arbitrary b) => Arbitrary (Tupley a b) where
    arbitrary = do
        a <- arbitrary
        b <- arbitrary
        return (Tupley a b)

In [8]:
instance (Semigroup a, Semigroup b) => Semigroup (Tupley a b) where
    (<>) (Tupley a b) (Tupley a' b') = Tupley (a <> a') (b <> b')
    
instance (Monoid a, Monoid b) => Monoid (Tupley a b) where
    mempty = Tupley mempty mempty
    mappend = (<>)

In [9]:
instance Functor (Tupley a) where
    fmap f (Tupley a b) = Tupley a (f b)

In [10]:
instance Monoid a => Applicative (Tupley a) where
    pure = Tupley mempty
    (Tupley u f) <*> (Tupley v x) = Tupley (u `mappend` v)  (f x)

In [11]:
instance (Eq a, Eq b) => EqProp (Tupley a b) where
    (=-=) = eq

In [12]:
trigger :: Tupley String (String, Int, Char)
trigger = undefined

verboseBatch (applicative trigger)


applicative:
  identity:     +++ OK, passed 500 tests.
  composition:  +++ OK, passed 500 tests.
  homomorphism: +++ OK, passed 500 tests.
  interchange:  +++ OK, passed 500 tests.
  functor:      +++ OK, passed 500 tests.

In [13]:
f :: Tupley String String -> Bool
f = applicativeId

quickCheck f

+++ OK, passed 100 tests.

In [14]:
c = applicativeCompose (++ "Hi") (++ " Bye!")
li x = c (x :: Tupley String String)

quickCheck li

+++ OK, passed 100 tests.

In [15]:
ah = applicativeHomomorphism (Tupley "" (++ "Hi")) (++ "Hi")
li x = ah (x :: String)

quickCheck li

+++ OK, passed 100 tests.

In [16]:
ai = applicativeInterchange (Tupley "" (++ "Hi"))
li x = ai (x :: String)

quickCheck li

+++ OK, passed 100 tests.

<h4>TypeCheck Exercises</h4>

In [17]:
added :: Maybe Integer
added = fmap (+3) (lookup 3 $ zip [1, 2, 3] [4, 5, 6])
added == Just 9

True

In [18]:
y :: Maybe Integer
y = lookup 3 $ zip [1, 2, 3] [4, 5, 6]

z :: Maybe Integer
z = lookup 2 $ zip [1, 2, 3] [4, 5, 6]

tupled :: Maybe (Integer, Integer)
tupled = (,) <$> y <*> z

tupled == Just (6, 5)

True

In [19]:
x :: Maybe Int
x = elemIndex 3 [1, 2, 3, 4, 5]

y :: Maybe Int
y = elemIndex 4 [1, 2, 3, 4, 5]

max' :: Int -> Int -> Int
max' = max

maxed :: Maybe Int
maxed = liftA2 max' x y

maxed == Just 3

True

In [20]:
xs = [1, 2, 3]
ys = [4, 5, 6]

x :: Maybe Integer
x = lookup 3 $ zip xs ys

y :: Maybe Integer
y = lookup 2 $ zip xs ys

summed :: Maybe Integer
summed = pure sum <*> ((,) <$> x <*> y)

summed == Just 5

True

<h4>Identity</h4>

In [21]:
newtype Identity a = Identity a deriving (Eq, Ord, Show)

In [22]:
instance Arbitrary a => Arbitrary (Identity a) where
    arbitrary = do
        x <- arbitrary
        return (Identity x)
    
instance Eq a => EqProp (Identity a) where
    (=-=) = eq

In [23]:
instance Functor Identity where
    fmap f (Identity a) = Identity (f a)
    
instance Applicative Identity where
    pure = Identity
    (<*>) (Identity f) (Identity a) = Identity (f a)

In [24]:
xs = [1, 2, 3]
ys = [9, 9, 9]

const <$> xs <*> ys

[1,1,1,2,2,2,3,3,3]

In [25]:
const <$> Identity xs <*> Identity ys

Identity [1,2,3]

In [26]:
trigger :: Identity (String, String, String)
trigger = undefined

quickBatch (applicative trigger)


applicative:
  identity:     +++ OK, passed 500 tests.
  composition:  +++ OK, passed 500 tests.
  homomorphism: +++ OK, passed 500 tests.
  interchange:  +++ OK, passed 500 tests.
  functor:      +++ OK, passed 500 tests.

<h4>Constant</h4>

In [27]:
newtype Constant a b = Constant { getConstant :: a } deriving (Eq, Ord, Show)

In [28]:
instance Arbitrary a => Arbitrary (Constant a b) where
    arbitrary = do
        a <- arbitrary
        return (Constant a)
    
instance Eq a => EqProp (Constant a b) where
    (=-=) = eq

In [29]:
instance Functor (Constant a) where
    fmap _ (Constant a) = Constant a

In [30]:
instance Semigroup a => Semigroup (Constant a b) where
    (<>) (Constant a) (Constant a') = Constant (a <> a')

instance Monoid a => Monoid (Constant a b) where
    mempty = Constant mempty
    mappend = (<>)

In [31]:
instance Monoid a => Applicative (Constant a) where
    pure = mempty
    (<*>) (Constant a) (Constant a') = Constant (a <> a')

In [32]:
f = Constant (Sum 1)
g = Constant (Sum 2)

In [33]:
trigger :: Constant (String, String, String) (String, String, String)
trigger = undefined

verboseBatch (applicative trigger)


applicative:
  identity:     +++ OK, passed 500 tests.
  composition:  +++ OK, passed 500 tests.
  homomorphism: +++ OK, passed 500 tests.
  interchange:  +++ OK, passed 500 tests.
  functor:      +++ OK, passed 500 tests.

<h4>Safe Constructors Name</h4>

In [34]:
validateLength :: Int -> String -> Maybe String
validateLength maxLen s = if (length s > maxLen) then Nothing else Just s

In [35]:
newtype Name = Name String deriving (Eq, Show)

newtype Address = Address String deriving (Eq, Show)

In [36]:
mkName :: String -> Maybe Name
mkName s = fmap Name (validateLength 32 s)

In [37]:
mkAddress :: String -> Maybe Address
mkAddress a = fmap Address (validateLength 100 a)

In [38]:
data Person = Person Name Address deriving (Eq, Show)

In [39]:
mkPerson :: String -> String -> Maybe Person
mkPerson n a = Person <$> mkName n <*> mkAddress a

In [40]:
mkPerson "Rohit" "234567890123456789012345678901234567890123456789012345678901234567890123456789012345678901234567890"

Just (Person (Name "Rohit") (Address "234567890123456789012345678901234567890123456789012345678901234567890123456789012345678901234567890"))

<h4>Safe Constructors Cow</h4>

In [41]:
data Cow = Cow { name:: String, age:: Int, weight:: Int } deriving (Eq, Show)

In [42]:
noEmpty :: String -> Maybe String
noEmpty "" = Nothing
noEmpty str = Just str

In [43]:
noNegative :: Int -> Maybe Int
noNegative n 
    | n >= 0     = Just n
    | otherwise  = Nothing

In [44]:
cowConstructor :: String -> Int -> Int -> Maybe Cow
cowConstructor n a w = Cow <$> noEmpty n <*> noNegative a <*> noNegative w 

In [45]:
cowConstructor "Rohit" (-1) 2

Nothing

<h4>Listy</h4>

In [46]:
data Listy a = Nil | Cons {head':: a, tail':: Listy a} deriving (Eq, Show)

instance Arbitrary a => Arbitrary (Listy a) where
    arbitrary = frequency [(1, pure Nil), (5, Cons <$> arbitrary <*> arbitrary)]
    
instance Eq a => EqProp (Listy a) where
    (=-=) = eq

In [47]:
instance Semigroup (Listy a) where
    (<>) Nil ys = ys
    (<>) (Cons x xs) ys = Cons x (xs <> ys)
    
instance Monoid (Listy a) where
    mempty = Nil
    mappend = (<>)

In [48]:
instance Functor Listy where
    fmap _ Nil = Nil
    fmap f (Cons x xs) = Cons (f x) (fmap f xs)

In [49]:
instance Applicative Listy where
    pure x = Cons x Nil
    (<*>) _ Nil = Nil
    (<*>) Nil _ = Nil
    (<*>) (Cons f fs) x = (f <$> x) <> (fs <*> x)

In [50]:
trigger :: Listy (String, String, String)
trigger = undefined

verboseBatch (applicative trigger)


applicative:
  identity:     +++ OK, passed 500 tests.
  composition:  +++ OK, passed 500 tests.
  homomorphism: +++ OK, passed 500 tests.
  interchange:  +++ OK, passed 500 tests.
  functor:      +++ OK, passed 500 tests.

<h4>ZipListy</h4>

In [51]:
newtype ZipListy a = ZipListy { getZipListy :: [a] } deriving (Eq, Show)

In [52]:
instance Arbitrary a => Arbitrary (ZipListy a) where
    arbitrary = ZipListy <$> arbitrary

In [53]:
instance Semigroup a => Semigroup (ZipListy a) where
    (<>) = 

<interactive>:2:12: error:
    • Could not deduce (Applicative ZipListy) arising from a use of ‘liftA2’
      from the context: Semigroup a bound by the instance declaration at <interactive>:1:10-46
    • In the expression: liftA2 (<>)
      In an equation for ‘<>’: (<>) = liftA2 (<>)
      In the instance declaration for ‘Semigroup (ZipListy a)’